In [1]:
file_path="/home/avnish/complaints.json"

In [2]:
from finance_complaint.entity.spark_manager import spark_session

22/09/13 14:02:11 WARN Utils: Your hostname, avnish-yadav resolves to a loopback address: 127.0.1.1; using 192.168.3.35 instead (on interface wlo1)
22/09/13 14:02:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/13 14:02:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark_session.read.json(file_path)

In [4]:
def updateattr(df):
    for colums in df.columns:
        setattr(df,colums,colums)
    

In [5]:
updateattr(df)

In [6]:
df.columns

['_corrupt_record',
 'company',
 'company_public_response',
 'company_response',
 'complaint_id',
 'complaint_what_happened',
 'consumer_consent_provided',
 'consumer_disputed',
 'date_received',
 'date_sent_to_company',
 'issue',
 'product',
 'state',
 'sub_issue',
 'sub_product',
 'submitted_via',
 'tags',
 'timely',
 'zip_code']

In [6]:
required_columns=["consumer_consent_provided","company_public_response","complaint_what_happened","company_response","company_public_response","issue","timely","tags","submitted_via","consumer_disputed"]

In [7]:
df=df.select(required_columns)

In [8]:
from pyspark.sql.functions import col

In [9]:
updateattr(df)

In [10]:
df=df.filter(col(df.consumer_disputed).isNotNull())

In [11]:
updateattr(df)

In [12]:
df.count()

2919299

In [15]:
df,rdf = df.randomSplit([0.1,1-0.1])

In [16]:
df.count()

290209

In [17]:
rdf.groupby(df.consumer_disputed).count().show()

+-----------------+-------+
|consumer_disputed|  count|
+-----------------+-------+
|               No| 558019|
|              N/A|1937445|
|              Yes| 133626|
+-----------------+-------+



In [21]:
df.groupby(df.consumer_disputed).count().show()

+-----------------+------+
|consumer_disputed| count|
+-----------------+------+
|               No| 61951|
|              N/A|214643|
|              Yes| 14670|
+-----------------+------+



In [18]:
pdf = df.toPandas()

In [23]:
pdf.consumer_disputed.value_counts()

N/A    214643
No      61951
Yes     14670
Name: consumer_disputed, dtype: int64

In [24]:
pdf.columns

Index(['consumer_consent_provided', 'company_public_response',
       'complaint_what_happened', 'company_response',
       'company_public_response', 'issue', 'timely', 'tags', 'submitted_via',
       'consumer_disputed'],
      dtype='object')

In [25]:
import pandas as pd

In [26]:
pd.DataFrame(pdf.consumer_consent_provided.value_counts())

,consumer_consent_provided
Consent provided,103588
Consent not provided,100222
N/A,71174
,8216
Other,7572
Consent withdrawn,492


In [33]:
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
input_feature = pdf[['consumer_consent_provided']]

In [39]:
ohe = OneHotEncoder()

In [40]:
x = ohe.fit_transform(input_feature)

In [44]:
x = x.toarray()

In [30]:
from sklearn.preprocessing import LabelEncoder

In [46]:
le = LabelEncoder()

In [51]:
y = le.fit_transform(pdf.consumer_disputed)

In [61]:
rfc = RandomForestClassifier()

In [49]:
rfc.fit(x,y)

RandomForestClassifier()

In [29]:
yhat=rfc.predict(x)

NameError: name 'rfc' is not defined

In [28]:
from sklearn.metrics import confusion_matrix,f1_score,recall_score

In [54]:
confusion_matrix(y,yhat)

array([[190438,  24205,      0],
       [ 23809,  38142,      0],
       [  5843,   8827,      0]])

In [57]:
f1_score(y,yhat,average="weighted")

0.7675215441217812

In [58]:
recall_score(y,yhat,average="weighted")

0.7847863107009448

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
X_train, X_test, y_train, y_test = train_test_split(x,y)

In [62]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [63]:
yhat_train = rfc.predict(X_train)
yhat_test = rfc.predict(X_test)

In [64]:
f1_score(y_train,yhat_train,average="weighted")

0.7678992401154284

In [65]:
f1_score(y_test,yhat_test,average="weighted")

0.7663847317662271

In [66]:
recall_score(y_train,yhat_train,average="weighted")

0.785230352303523

In [67]:
recall_score(y_test,yhat_test,average="weighted")

0.7834541858932103

In [93]:
d=pdf.company_public_response

In [94]:
d.columns=["company_public_response","NO"]

In [95]:
d.columns = ["company_public_response","NO"]

In [100]:
d.company_public_response.unique()

array(['',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       'Company believes complaint is the result of an isolated error',
       'Company believes complaint represents an opportunity for improvement to better serve consumers',
       'Company believes the complaint is the result of a misunderstanding',
       "Company believes the complaint provided an opportunity to answer consumer's questions",
       "Company can't verify or dispute the facts in the complaint",
       'Company chooses not to provide a public response',
       'Company disputes the facts presented in the complaint',
       'Company believes complaint relates to a discontinued policy or procedure'],
      dtype=object)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [102]:
tf = TfidfVectorizer()

In [104]:
x=tf.fit_transform(d.company_public_response)

In [107]:
data = x.toarray()

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [111]:
rfc = RandomForestClassifier()

In [112]:
y

array([0, 0, 0, ..., 0, 1, 0])

In [113]:
X_train, X_test, y_train, y_test = train_test_split(data,y)

In [114]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [115]:
yhat_train = rfc.predict(X_train)
yhat_test = rfc.predict(X_test)

In [116]:
f1_score(y_train,yhat_train,average="weighted")

0.6587785655822808

In [117]:
f1_score(y_test,yhat_test,average="weighted")

0.6598380437969992

In [118]:
recall_score(y_train,yhat_train,average="weighted")

0.7514145242803779

In [119]:
recall_score(y_test,yhat_test,average="weighted")

0.7523621182157767

In [122]:
pd.DataFrame(pdf.company_response.value_counts())

,company_response
Closed with explanation,228736
Closed with non-monetary relief,40445
Closed with monetary relief,11829
In progress,5200
Closed without relief,1840
Closed,1716
Untimely response,960
Closed with relief,538


In [23]:
from sklearn.metrics import precision_score

In [39]:
def test_feature(df,input_column,output_column="consumer_disputed"):
    x = df[[input_column]]
    ohe = OneHotEncoder()
    x=ohe.fit_transform(x)
    x=x.toarray()
    lb = LabelEncoder()
    y = lb.fit_transform(df[output_column])
    
    X_train, X_test, y_train, y_test = train_test_split(x,y)
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    yhat_train = rfc.predict(X_train)
    yhat_test = rfc.predict(X_test)
    fi_score_train = f1_score(y_train,yhat_train,average="weighted")
    print(f"f1 train score: {fi_score_train}")
    test_score = f1_score(y_test,yhat_test,average="weighted")
    print(f"f1 test score: {test_score}")
    rs = recall_score(y_train,yhat_train,average="weighted")
    print(f"recall train score: {rs}")
    recall_score(y_train,yhat_train,average="weighted")
    print(f"recall test score: {rs}")
    rs = precision_score(y_train,yhat_train,average="weighted")
    print(f"precision train score: {rs}")
    rs = precision_score(y_train,yhat_train,average="weighted")
    print(f"precision test score: {rs}")


In [131]:
test_feature(pdf,'company_response','consumer_disputed')

f1 train score: 0.6496484952427628
f1 test score: 0.6538442940695111
recall train score: 0.7469145975243536
recall test score: 0.7469145975243536
precision train score: 0.713972878294822
precision test score: 0.713972878294822


/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
drop=["complaint_what_happened","timely"]

In [22]:
def test_feature_tfid(df,input_column,output_column="consumer_disputed"):
    x = df[[input_column]]
    print(x.shape)
    tf = TfidfVectorizer()
    x=tf.fit_transform(x[input_column])
    x=x.toarray()
    lb = LabelEncoder()
    y = lb.fit_transform(df[output_column])
    print(y.shape)
    print(x.shape,y.shape)
    X_train, X_test, y_train, y_test = train_test_split(x,y)
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    yhat_train = rfc.predict(X_train)
    yhat_test = rfc.predict(X_test)
    fi_score_train = f1_score(y_train,yhat_train,average="weighted")
    print(f"f1 train score: {fi_score_train}")
    test_score = f1_score(y_test,yhat_test,average="weighted")
    print(f"f1 test score: {test_score}")
    rs = recall_score(y_train,yhat_train,average="weighted")
    print(f"recall train score: {rs}")
    recall_score(y_train,yhat_train,average="weighted")
    print(f"recall test score: {rs}")
    rs = precision_score(y_train,yhat_train,average="weighted")
    print(f"precision train score: {rs}")
    rs = precision_score(y_train,yhat_train,average="weighted")
    print(f"precision test score: {rs}")


In [153]:
pdf.columns

Index(['consumer_consent_provided', 'company_public_response',
       'complaint_what_happened', 'company_response',
       'company_public_response', 'issue', 'timely', 'tags', 'submitted_via',
       'consumer_disputed'],
      dtype='object')

In [154]:
test_feature_tfid(pdf,'issue')

(291264, 1)
(291264,)
(291264, 238) (291264,)
f1 train score: 0.9108372995447654
f1 test score: 0.9125899598882266
recall train score: 0.933251849410386
recall test score: 0.933251849410386
precision train score: 0.9419461712530541
precision test score: 0.9419461712530541


0         Advertising and marketing, including promotion...
1         Advertising and marketing, including promotion...
2         Applying for a mortgage or refinancing an exis...
3         Applying for a mortgage or refinancing an exis...
4         Applying for a mortgage or refinancing an exis...
                                ...                        
290204                 Incorrect information on your report
290205                 Incorrect information on your report
290206              Account opening, closing, or management
290207                 Incorrect information on your report
290208                 Incorrect information on your report
Name: issue, Length: 290209, dtype: object

In [ ]:
test_feature_tfid(pdf,'complaint_what_happened')

(291264, 1)


In [19]:
pdf.columns

Index(['consumer_consent_provided', 'company_public_response',
       'complaint_what_happened', 'company_response',
       'company_public_response', 'issue', 'timely', 'tags', 'submitted_via',
       'consumer_disputed'],
      dtype='object')

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
test_feature_tfid(pdf,'timely')

(290209, 1)
(290209,)
(290209, 2) (290209,)
f1 train score: 0.6226692722998821
f1 test score: 0.6246814956453622
recall train score: 0.734948726430698
recall test score: 0.734948726430698
precision train score: 0.540149630482105
precision test score: 0.540149630482105


/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
test_feature_tfid(pdf,'tags')

(290209, 1)
(290209,)
(290209, 3) (290209,)
f1 train score: 0.6237985879088424
f1 test score: 0.6212939549306051
recall train score: 0.7357940971073621
recall test score: 0.7357940971073621
precision train score: 0.5413929533380382
precision test score: 0.5413929533380382


/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
test_feature(pdf,'tags')

f1 train score: 0.6240625828791486
f1 test score: 0.6205030120200608
recall train score: 0.7359916565589738
recall test score: 0.7359916565589738
precision train score: 0.5416837185244224
precision test score: 0.5416837185244224


/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
test_feature(pdf,'submitted_via')

f1 train score: 0.6919492959412933
f1 test score: 0.6972809082137885
recall train score: 0.7404757966699747
recall test score: 0.7404757966699747
precision train score: 0.6709008659038586
precision test score: 0.6709008659038586


/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/finance_complaint/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
consumer_consent_provided: ONEHOT,
COMPANY_PUBLIC_RESPONSE = TFIDF
company_response=ONE_HOT
issue=TFIDF

In [13]:
from pyspark.ml.feature import Tokenizer

In [14]:
from pyspark.mllib.feature import Word2Vec

In [15]:
word2vec = Word2Vec()

MapPartitionsRDD[36] at javaToPython at NativeMethodAccessorImpl.java:0

In [19]:
word2vec.fit(df.select(col(df.issue)).rdd.collect())

TypeError: data should be an RDD of list of string

In [24]:
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()
tf = hashingTF.transform(df[[df.issue]].rdd)

In [27]:
tf.cache()

PythonRDD[49] at RDD at PythonRDD.scala:53

In [28]:
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [30]:
tfidf.take(1)

[SparseVector(1048576, {670126: 1.4122})]